## Processing Wikisource
- pyhon 10.3 버전 가상환경 구축 필요.
- 필요한 라이브러리 설치. (wikiextractor==3.0.6 gdown==5.2.0 사용)

In [11]:
!pip install -q wikiextractor gdown
!pip install pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 3.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [1]:
# 필요 패키지 load
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import glob, os
import re

In [2]:
# 위키문헌 xml 원본 파일 다운로드
# https://dumps.wikimedia.org/kowikisource/20240901/kowikisource-20240901-pages-articles.xml.bz2
#!wget https://dumps.wikimedia.org/kowikisource/20241101/kowikisource-20241101-pages-articles.xml.bz2
!wget https://dumps.wikimedia.org/kowikisource/20250801/kowikisource-20250801-pages-articles.xml.bz2

--2025-08-24 13:27:46--  https://dumps.wikimedia.org/kowikisource/20250801/kowikisource-20250801-pages-articles.xml.bz2
dumps.wikimedia.org (dumps.wikimedia.org) 해석 중... 208.80.154.71
다음으로 연결 중: dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 154289165 (147M) [application/octet-stream]
저장 위치: `kowikisource-20250801-pages-articles.xml.bz2.1'

kowikisource-202508 100%[===================>] 147.14M  1.18MB/s    /  2m 7s   

2025-08-24 13:29:54 (1.16 MB/s) - `kowikisource-20250801-pages-articles.xml.bz2.1' 저장함 [154289165/154289165]



In [3]:
# json 파일로 변환
!python -m wikiextractor.WikiExtractor --json kowikisource-20250801-pages-articles.xml.bz2

INFO: Preprocessing 'kowikisource-20250801-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Loaded 1588 templates in 28.8s
INFO: Starting page extraction from kowikisource-20250801-pages-articles.xml.bz2.
INFO: Using 7 extract processes.
INFO: Finished 7-process extraction of 31062 articles in 44.8s (694.0 art/s)


In [4]:
# text폴터 내 json 파일들을 읽어와서 하나의 DataFrame으로 합치기
files = glob.glob("text/**/*", recursive=True)  # 모든 출력 파일 경로
dfs = []
for f in files:
    if os.path.isfile(f):
        dfs.append(pd.read_json(f, lines=True))

df = pd.concat(dfs, ignore_index=True)
print(df.head())

      id  revid                                         url          title  \
0  29759   5442  https://ko.wikisource.org/wiki?curid=29759          애인/하권   
1  29775  15531  https://ko.wikisource.org/wiki?curid=29775             고향   
2  29779  15531  https://ko.wikisource.org/wiki?curid=29779  파리에서 본 것 늣긴 것   
3  29782   8771  https://ko.wikisource.org/wiki?curid=29782          동물 시집   
4  29783   3009  https://ko.wikisource.org/wiki?curid=29783       동물 시집/나비   

                                                text  
0  人魚[인어]와 유모레스크.\n이튿날 아침, 부산역에서 내렸을 때, 석란은 가벼운 현...  
1                                                     
2  내가 巴里[파리]에 있을 적 일이다 주인[主人]집에서 친구 哲學博士[철학박사]를 主...  
3                                                     
4  &lt;poem&gt;\n비바람 험살궂게 거쳐 간 추녀 밑 \n날개 찢어진 늙은 노...  


In [5]:
df

,id,revid,url,title,text
0,29759,5442,https://ko.wikisource.org/wiki?curid=29759,애인/하권,"人魚[인어]와 유모레스크.\n이튿날 아침, 부산역에서 내렸을 때, 석란은 가벼운 현..."
1,29775,15531,https://ko.wikisource.org/wiki?curid=29775,고향,
2,29779,15531,https://ko.wikisource.org/wiki?curid=29779,파리에서 본 것 늣긴 것,내가 巴里[파리]에 있을 적 일이다 주인[主人]집에서 친구 哲學博士[철학박사]를 主...
3,29782,8771,https://ko.wikisource.org/wiki?curid=29782,동물 시집,
4,29783,3009,https://ko.wikisource.org/wiki?curid=29783,동물 시집/나비,&lt;poem&gt;\n비바람 험살궂게 거쳐 간 추녀 밑 \n날개 찢어진 늙은 노...
...,...,...,...,...,...
31056,78868,15531,https://ko.wikisource.org/wiki?curid=78868,할미꽃,"할미꽃 全 一幕\n인물\n시일\n장경(場景)\n丁, 어떡하면 좋아요, 다른 과에서는..."
31057,78872,15531,https://ko.wikisource.org/wiki?curid=78872,아인슈타인 박사와 탐정소설,「아인슈타인 博士와 探偵小說」\n金來成\n自然現象과 探偵小說.\n「物理學의 發展」“...
31058,78881,15531,https://ko.wikisource.org/wiki?curid=78881,최근 문예이론의 신전개와 그 경향,─ 社會學的 及 文學史的 考察\n一.\n一九二九年 以後부터 나는 나의 『카푸』에 關...
31059,78888,15531,https://ko.wikisource.org/wiki?curid=78888,치악산,


### 시 행별/작품별 추출하는 함수

In [6]:
def process_getPoemLines(data, title): # 위키문헌에서 시 본문 행단위 추출
    # 불필요한 인자 제거
    # filtered_list = [item for item in data if item not in [None, "", " ", [], {}, '&lt;poem&gt;', '&lt;/poem&gt;','&lt;poem&gt; ']]

    # 제거할 패턴 (HTML escape된 <poem>, </poem>, 변형 포함)
    poem_pattern = re.compile(r'&lt;/?(poem|pages|doc)\b[^&]*?&gt;', re.IGNORECASE)
    # 문자열 필터링: 빈 문자열 및 <poem> 태그류 제거
    filtered_list = [
        item for item in data
        if isinstance(item, str)                          # 문자열만 검사
        and item.strip() not in ("",)                     # 빈 문자열 제거
        and not poem_pattern.fullmatch(item.strip())      # <poem> 태그류 제거
    ]
    
    body = filtered_list[0:]

    # DataFrame 생성: 본문 열이 먼저 오도록
    df = pd.DataFrame({'본문': body, '제목': [title] * len(body)})

    return df

In [7]:
def process_getPoemFullLines(data, title): # 위키문헌에서 시 본문 전체 추출
    # 불필요한 인자 제거
    # cleaned_text = data.replace('&lt;poem&gt;', '').replace('&lt;/poem&gt;', '').strip()
    cleaned_text = re.sub(r'&lt;/?(poem|pages|doc)\b[^&]*?&gt;', '', data).strip()
    # 텍스트를 줄 단위로 나누기
    lines = cleaned_text.strip().split('\n')
    body = '\n'.join(lines[0:])  # 나머지는 본문

    # DataFrame 생성
    df = pd.DataFrame({'제목': [title], '본문': [body]})

    return df

### index를 검색하여 전체 데이터에서 한가지 작품만 행/작품단위로 데이터 추출

In [8]:
def getOnlyOneresult_with_title(df, results_df, process_function, title_to_search, author, index):
    # 제목에 해당하는 텍스트 검색
    title_to_search.replace("_", " ")
    filtered_text = df[df['title'].str.contains(title_to_search, na=False)]
    if filtered_text.empty:
        raise ValueError(f"'{title_to_search}'에 해당하는 데이터가 없습니다.")
    # 'text' 값 추출
    text_value = filtered_text.iloc[index]['text']

    # 주어진 함수로 제목(title), 본문(text) 처리
    processed_result = process_function(text_value.split('\n'),filtered_text['title'].values[0])

    # 처리 결과를 기존 DataFrame에 추가
    updated_results = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)

    # 새로운 열 추가 (고정된 값)
    updated_results['저자'] = author

    return updated_results

In [9]:
def getOnlyOneFullTextresult_with_title(df, results_df, process_function, title_to_search, author, index):
    # 제목에 해당하는 텍스트 검색
    title_to_search.replace("_", " ")
    filtered_text = df[df['title'].str.contains(title_to_search, na=False)]
    if filtered_text.empty:
        raise ValueError(f"'{title_to_search}'에 해당하는 데이터가 없습니다.")

    # 'text' 값 추출
    full_text_value = filtered_text.iloc[index]['text']

    # 주어진 함수로 제목(title), 본문(text) 처리
    processed_result = process_function(full_text_value, filtered_text['title'].values[0])

    # 처리 결과를 기존 DataFrame에 추가
    updated_results = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)
    # 새로운 열 추가 (고정된 값)
    updated_results['저자'] = author

    return updated_results

In [10]:
#검색하고자 하는 제목과 자료 순서 입력.
title = '진달래꽃'
index = 0
author = '김소월'
#바르게 검색되었는지 확인.
check = df[df['title'].str.contains(title)]
check

,id,revid,url,title,text
3479,34605,951,https://ko.wikisource.org/wiki?curid=34605,진달래꽃 (시집)/님에게,&lt;poem&gt;\n한때는 많은 날을 당신 생각에\n밤까지 새운 일도 없지 않...
3480,34606,290727,https://ko.wikisource.org/wiki?curid=34606,진달래꽃 (시집)/산 위에,현대어 \n&lt;poem&gt;\n산 위에\n산(山) 위에 올라서서 바라다보면\n...
3481,34607,951,https://ko.wikisource.org/wiki?curid=34607,진달래꽃 (시집)/산,&lt;poem&gt;\n산(山)새도 오리나무\n위에서 운다\n산(山)새는 왜 우노...
3482,34608,951,https://ko.wikisource.org/wiki?curid=34608,진달래꽃 (시집)/실제 1,&lt;poem&gt;\n동무들 보십시오 해가 집니다\n해지고 오늘날은 가노랍니다\...
3483,34609,951,https://ko.wikisource.org/wiki?curid=34609,진달래꽃 (시집)/실제 2,&lt;poem&gt;\n이 가람과 저 가람이 모두 처흘러\n그 무엇을 뜻하는고?\...
...,...,...,...,...,...
25362,1519,204518,https://ko.wikisource.org/wiki?curid=1519,진달래꽃 (시집)/금잔디,"&lt;poem&gt;\n잔디,\n잔디,\n금잔디,\n심심산천에 붙는 불은\n가신 ..."
25363,1521,951,https://ko.wikisource.org/wiki?curid=1521,진달래꽃 (시집)/엄마야 누나야,"&lt;poem&gt;\n엄마야 누나야 강변 살자,\n뜰에는 반짝는 금모래빛,\n뒷..."
25364,1522,951,https://ko.wikisource.org/wiki?curid=1522,진달래꽃 (시집)/산유화,현대어 \n&lt;poem&gt;\n산유화\n산에는 꽃 피네\n꽃이 피네\n갈 봄 ...
25365,1523,951,https://ko.wikisource.org/wiki?curid=1523,진달래꽃 (시집)/왕십리,&lt;poem&gt;\n비가 온다\n오누나\n오는 비는\n올지라도 한 닷새 왔으면...


In [11]:
#빈 프레임 만들기 (열은 본문과 title)
one_poem_lines = pd.DataFrame(columns=['본문', '제목'])

In [12]:
# 작품 한개씩 행 추출 (데이터에 포함된 작품이름과, 작가 이름을 검색.) #여러개를 쌓을 수 있다
one_poem_lines = getOnlyOneresult_with_title(df, one_poem_lines, process_getPoemLines, title, author, index) #검색어, 작가 이름 입력
one_poem_lines.iloc[-1] # 마지막줄 확인

본문    당신은 잊어버린 설움이외다
제목     진달래꽃 (시집)/님에게
저자               김소월
Name: 11, dtype: object

In [14]:
one_full_poem = pd.DataFrame(columns=['본문', '제목'])

In [15]:
# 작품 한개씩 행 추출 (데이터에 포함된 작품이름과, 작가 이름을 검색.) #여러개를 쌓을 수 있다
one_full_poem = getOnlyOneFullTextresult_with_title(df, one_full_poem, process_getPoemFullLines, title, author, index) #검색어, 작가 이름 입력
one_full_poem.iloc[-1] # 마지막줄 확인

본문    한때는 많은 날을 당신 생각에\n밤까지 새운 일도 없지 않지만\n아직도 때마다는 당...
제목                                        진달래꽃 (시집)/님에게
저자                                                  김소월
Name: 0, dtype: object

In [17]:
one_poem_lines.to_csv('kowikisource_poem_lines.csv', index=False, encoding='utf-8-sig')
one_full_poem.to_csv('kowikisource_full_poem.csv', index=False, encoding='utf-8-sig')

### title에 해당하는 복수의 데이터를 행 단위/ 작품 단위로 추출

In [18]:
def update_results_with_title(df, results_df, process_function, title_to_search, author):

    # 제목에 해당하는 텍스트 검색
    title_to_search.replace("_", " ")
    filtered_text = df[df['title'].str.contains(title_to_search, na=False)]
    if filtered_text.empty:
        raise ValueError(f"'{title_to_search}'에 해당하는 데이터가 없습니다.")

    # 조건 반복문으로 'doc' 처리
    for index, row in filtered_text.iterrows():
        if 'text' in row:  # 'doc' 열이 존재하는지 확인
            # 'doc' 값 추출
            text_value = row['text']
            title =  row['title']

            # 텍스트 처리
            processed_result = process_function(text_value.split('\n'), title)

            # 처리 결과를 기존 DataFrame에 추가
            results_df = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)

            # # 새로운 열 추가 (고정된 값)
            results_df['저자'] = author
        else:
            print(f"Row {index}에 'text' 값이 없습니다. 건너뜁니다.")
    return results_df

In [19]:
def update_FullTextResults_with_title(df, results_df, process_function, title_to_search, author):

    # 제목에 해당하는 텍스트 검색
    title_to_search.replace("_", " ")
    filtered_text = df[df['title'].str.contains(title_to_search, na=False)]
    if filtered_text.empty:
        raise ValueError(f"'{title_to_search}'에 해당하는 데이터가 없습니다.")


    # 조건 반복문으로 'doc' 처리
    for index, row in filtered_text.iterrows():
        if 'text' in row:  # 'doc' 열이 존재하는지 확인
            # 'doc' 값 추출
            text_value = row['text']
            title =  row['title']

            # 텍스트 처리
            processed_result = process_function(text_value, title)

            # 처리 결과를 기존 DataFrame에 추가
            results_df = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)

            # # 새로운 열 추가 (고정된 값)
            results_df['저자'] = author
        else:
            print(f"Row {index}에 'text' 값이 없습니다. 건너뜁니다.")
    return results_df

In [20]:
# 검색 데이터 확인.
title = '님의 침묵'
author = '한용운'
check = df[df['title'].str.contains(title)]
check

,id,revid,url,title,text
1369,24999,15531,https://ko.wikisource.org/wiki?curid=24999,님의 침묵/자유정조,"自由貞操.\n&lt;poem&gt;\n&lt;pages index=""CNTS-000..."
1370,25001,15531,https://ko.wikisource.org/wiki?curid=25001,님의 침묵/차라리,"차라리.\n&lt;poem&gt;\n&lt;pages index=""CNTS-0007..."
1371,25002,15531,https://ko.wikisource.org/wiki?curid=25002,님의 침묵/당신이 아니더면,"당신이아니더면.\n&lt;poem&gt;\n&lt;pages index=""CNTS-..."
1372,25003,15531,https://ko.wikisource.org/wiki?curid=25003,님의 침묵/잠 없는 꿈,"잠업는ᄭᅮᆷ.\n&lt;poem&gt;\n&lt;pages index=""CNTS-0..."
1373,25004,15531,https://ko.wikisource.org/wiki?curid=25004,님의 침묵/진주,"眞珠.\n&lt;poem&gt;\n&lt;pages index=""CNTS-00076..."
...,...,...,...,...,...
26924,12458,15531,https://ko.wikisource.org/wiki?curid=12458,님의 침묵/당신 가신 때,"당신가신ᄯᅢ.\n&lt;poem&gt;\n&lt;pages index=""CNTS-0..."
26925,12459,15531,https://ko.wikisource.org/wiki?curid=12459,님의 침묵/당신은,"당신은.\n&lt;poem&gt;\n&lt;pages index=""CNTS-0007..."
27500,2239,7,https://ko.wikisource.org/wiki?curid=2239,한용운의 님의 침묵,
27907,1362,5442,https://ko.wikisource.org/wiki?curid=1362,님의 침묵/가갸날,"&lt;poem&gt;\n아아, 가갸날\n참되고 어질고 아름다와요\n축일(祝日) 제..."


In [21]:
#빈 프레임 지정. 열 지정.
poems_lines = pd.DataFrame(columns=['본문', '제목'])

In [22]:
poems_lines = update_results_with_title(df, poems_lines, process_getPoemLines, title, author)
poems_lines.iloc[-1] # 마지막줄 확인

본문    나는 해저문 벌판에서 돌아가는 길을 잃고 헤매는 어린 양(羊)이 그리워서 이 시(詩...
제목                                             님의 침묵/군말
저자                                                  한용운
Name: 1083, dtype: object

In [23]:
poems_full = pd.DataFrame(columns=['본문', '제목'])

In [24]:
poems_full= update_FullTextResults_with_title(df, poems_full, process_getPoemFullLines, title, author)
poems_full.iloc[-1] # 마지막줄 확인

본문    군말.\n\n현대어.\n\n「님」만 님이 아니라 그리운 것은 다 님이다 중생(衆生)...
제목                                             님의 침묵/군말
저자                                                  한용운
Name: 96, dtype: object

In [25]:
poems_lines.to_csv('한용운_님의침묵_lines.csv', index=False, encoding='utf-8-sig')
poems_full.to_csv('한용운_님의침묵_full.csv', index=False, encoding='utf-8-sig')